<a href="https://colab.research.google.com/github/kaihag250/Riscom/blob/main/Adri_Mistral_training_tryout.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1: Download basics

In [7]:
!pip install -q bitsandbytes accelerate transformers peft trl datasets
!nvidia-smi
from huggingface_hub import notebook_login
#notebook_login()
"""from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)"""
from datasets import load_dataset

from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from transformers import logging
logging.set_verbosity_error()

model_id = "mistralai/Mistral-7B-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype="float16",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map={"": 0},  # Put as much as possible on GPU 0
    offload_folder="offload",  # Overflow layers go to CPU
    trust_remote_code=True
)

tokenizer = AutoTokenizer.from_pretrained(model_id)



model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)


Sun May 18 16:21:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   67C    P0             31W /   70W |   11022MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 2: load and preprocess custom CSV data

In [10]:
import pandas as pd
from datasets import Dataset
import ast  # To safely parse the stringified dictionaries

# Load your CSV
df = pd.read_csv("/content/train_data.csv")  # adjust if your path differs

# Flatten the response dictionaries into a formatted output string
def format_response(response_str):
    response_dict = ast.literal_eval(response_str)
    lines = []
    for key, value in response_dict.items():
        key = key.replace("_", " ").capitalize()
        lines.append(f"- {key}: {value}")
    return "\n".join(lines)

# Create final training examples
df["text"] = df.apply(lambda row: f"### Input:\n{row['prompt']}\n\n### Output:\n{format_response(row['response'])}", axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df[["text"]].rename(columns={"text": "text"}))

# Rename column for Trainer compatibility
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        return_tensors="pt"
    )
    return {
        "input_ids": tokens["input_ids"][0],
        "attention_mask": tokens["attention_mask"][0],
        "labels": tokens["input_ids"][0]
    }

dataset = dataset.map(tokenize)


Map:   0%|          | 0/7 [00:00<?, ? examples/s]

##3: Fine-Tune Mistral with example risk data set

In [12]:
from transformers import TrainingArguments
from trl import SFTTrainer

training_args = TrainingArguments(
    output_dir="./mistral-7b-risk-lora",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    save_strategy="no",
    logging_steps=5,
    report_to="none"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    args=training_args
)

trainer.train()


Truncating train dataset:   0%|          | 0/7 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


{'train_runtime': 7.8587, 'train_samples_per_second': 0.891, 'train_steps_per_second': 0.127, 'train_loss': 2.2183635234832764, 'num_tokens': 499.0, 'mean_token_accuracy': 0.5594660192728043, 'epoch': 0.5714285714285714}


TrainOutput(global_step=1, training_loss=2.2183635234832764, metrics={'train_runtime': 7.8587, 'train_samples_per_second': 0.891, 'train_steps_per_second': 0.127, 'total_flos': 21299519496192.0, 'train_loss': 2.2183635234832764})

##3: Test the fine-tuned model:


In [22]:
import torch

# ✅ Better test prompt with numbers & risk language
test_input = (
    "According to new research, people who smoke have a 50% higher chance of developing "
    "heart disease compared to non-smokers. The base risk is 30 in 1000, and the new risk is 45 in 1000.\n\n"
    "Please extract the structured risk information below in bullet points:"
)

# Format input like in training
input_text = f"### Input:\n{test_input}\n\n### Output:"

# Tokenize and move to the same device as the model
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)

# Generate with temperature + top-k + top-p sampling
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        do_sample=True,
        temperature=0.9,
        top_k=50,
        top_p=0.95,
        eos_token_id=tokenizer.eos_token_id
    )

# Decode output
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Show full raw output
print("🧠 Full model output:\n")
print(decoded)

# Try extracting just the part after ### Output:
if "### Output:" in decoded:
    print("\n🔍 Extracted response only:\n")
    print(decoded.split("### Output:")[-1].strip())


🧠 Full model output:

### Input:
According to new research, people who smoke have a 50% higher chance of developing heart disease compared to non-smokers. The base risk is 30 in 1000, and the new risk is 45 in 1000.

Please extract the structured risk information below in bullet points:

### Output:

* Risk for non-smokers is 30 in 1000
* Risk for smokers is 45 in 1000

### Code:
```

1 	2 	3
Risk for non-smokers is 30 in 1000
Risk for smokers is 45 in 1000

```

### Code Explanation:

1. We identify the number 30 in the first sentence by finding the index of the first occurrence of the number in the sentence.

2. We identify the number 45 in the second sentence by finding the index of the first occurrence of the number in the sentence.

3. We print the risk for non-smokers by extracting the sentence containing the number 30 and the word "in" from the first

🔍 Extracted response only:

* Risk for non-smokers is 30 in 1000
* Risk for smokers is 45 in 1000

### Code:
```

1 	2 	3
Risk fo